In [1]:
import pandas as pd

In [2]:
%load_ext rpy2.ipython

# Question Engagement Analysis

## Select course and load data set

In [3]:
data_dir = '/Users/benny/data/L@S_2021'
course = 'microbiology'

In [4]:
data_set_filename = f'engagement_{course}.txt'
data_set = pd.read_csv( f'{data_dir}/{data_set_filename}', sep='\t' )

In [5]:
data_set

,student,question,course_page_number,unit_page_number,module_page_number,page_question_number,question_type,answered
0,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135e55,18,16,5,0,AG_FITB,1
1,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135e77,18,16,5,1,AG_FITB,1
2,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135e9d,18,16,5,2,AG_Matching,1
3,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135ead,18,16,5,3,AG_Matching,1
4,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135ebd,18,16,5,4,AG_Matching,1
...,...,...,...,...,...,...,...,...
71114,600bfe6a15f61a7c2756c0c9,5f2e4e8ae3a4d9432b13bfce,28,26,6,9,AG_Matching,0
71115,600bfe6a15f61a7c2756c0c9,5f2e4e8ae3a4d9432b13bff0,28,26,6,10,AG_FITB,0
71116,600bfe6a15f61a7c2756c0c9,5f2e4e8ae3a4d9432b13c016,28,26,6,11,AG_Matching,0
71117,600bfe6a15f61a7c2756c0c9,5f2e4e8ae3a4d9432b13c026,28,26,6,12,AG_Matching,0


## Mean engagement

In [6]:
data_set.groupby( 'question_type' ).mean().sort_values( by='answered', ascending=False )

,course_page_number,unit_page_number,module_page_number,page_question_number,answered
question_type,,,,,
HA_MC,43.078851,41.078851,4.456248,9.170459,0.692540
AG_Matching,51.977292,49.977292,4.186578,4.906856,0.669077
AG_FITB,46.576671,44.576671,4.448933,5.272404,0.618919
HA_MCMS,46.680868,44.680868,5.349678,11.062701,0.549035
HA_DnD_Table,44.718085,42.718085,7.797872,11.925532,0.471277
HA_Pulldown,57.174006,55.174006,6.077913,21.394422,0.422764
HA_DnD_Image,54.000000,52.000000,6.000000,27.500000,0.340944
HA_SnC,48.058331,46.058331,6.988286,13.410949,0.328951


## Regression model

In [7]:
%%R
library( lme4 )

R[write to console]: Loading required package: Matrix



Standardize the continuous variables.

In [8]:
for col in [ 'course_page_number', 'unit_page_number', 'module_page_number', 'page_question_number' ]:
    data_set[ col ] = ( data_set[ col ] - data_set[ col ].mean() ) / data_set[ col ].std()

In [9]:
data_set.to_csv( '/tmp/to_r.csv', index=False )

In [10]:
%%R
df <- read.csv( '/tmp/to_r.csv' )

In [11]:
%%R
lme.model <- glmer( answered ~ course_page_number + unit_page_number + module_page_number + page_question_number + question_type
                             + (1|student) + (1|question), family=binomial(link=logit),
                    data=df,
                    control=glmerControl( optimizer="bobyqa", optCtrl=list(maxfun=2e4) ) )
summary( lme.model )

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
answered ~ course_page_number + unit_page_number + module_page_number +  
    page_question_number + question_type + (1 | student) + (1 |      question)
   Data: df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 20000))

     AIC      BIC   logLik deviance df.resid 
 49784.4  49903.7 -24879.2  49758.4    71106 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-65.455  -0.314   0.081   0.374  16.373 

Random effects:
 Groups   Name        Variance Std.Dev.
 question (Intercept) 0.5537   0.7441  
 student  (Intercept) 9.0917   3.0153  
Number of obs: 71119, groups:  question, 1106; student, 99

Fixed effects:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -0.002364   0.314532  -0.008 0.994004    
course_page_number        -0.909442   0.025880 -35.141  < 2e-16 ***
module_page_num